Import Stuff

In [1]:
import ssl
import re


from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from umap import UMAP

from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datetime import datetime
from textblob import TextBlob
from datasets import load_dataset

import pickle
import importlib
import functions
import requests





c:\Users\Jacob\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from functions import *
importlib.reload(functions)

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

parties = ["CDU", "SPD", "FDP", "AFD", "LEFT", "GREENS"]
file_path = "C:/Users/Jacob/OneDrive/uni/MA WiSoz/Semester III/Computational Social Sciences/foPra/data/"


# Open the file in read mode
with open("API Keys/DeepL.txt", 'r', encoding='utf-8') as file:
    # Read the contents of the file
    api_key_deepl = file.read()

url = 'https://api-free.deepl.com/v2/translate'


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Edit Text documents

In [3]:
df_manifesto = pd.DataFrame()
for p in parties:
    new_df = pd.read_csv(file_path + "parties/"+ p + "_manifesto.csv")
    new_df["party"] = p  # Add a new column for the party name
    df_manifesto = pd.concat([df_manifesto, new_df], axis=0)


df_manifesto['date'] = pd.to_datetime(df_manifesto['date'], format='%Y%m')



In [4]:
split_date = datetime(1998, 9, 1)
df_manifesto = split_text_by_date(df_manifesto, split_date)

# Count occurrences of distinct values in the party column
party_counts = df_manifesto["party"].value_counts()

# Add a line that contains this count per party
df_manifesto["party_count"] = df_manifesto["party"].map(party_counts)
df_manifesto["cmp_code"] = pd.to_numeric(df_manifesto["cmp_code"], errors='coerce')
df_manifesto.index = range(1, len(df_manifesto) + 1)

# Delete entries that do not contain words in the text column
df_manifesto = df_manifesto[df_manifesto['text'].str.contains(r'\b\w+\b', na=False)]

# Ensure the 'text' column contains only strings and handle NaN values
df_manifesto["text"] = df_manifesto["text"].astype(str).fillna("")

df_manifesto_final = merge_text_rows(df_manifesto)

,text,cmp_code,eu_code,pos,manifesto_id,party,date,language,annotations,translation_en,party_count
0,Düsseldorfer Leitsätze vom 15. Juli 1949 Das ...,NaN,NaN,1,41521_194908,CDU,1949-08-01,german,False,False,17734
1,In der ersten Hälfte des Jahres 1948 war der T...,NaN,NaN,1,41521_194908,CDU,1949-08-01,german,False,False,17734
2,Produktion und Arbeitsmoral waren auf ein Mini...,NaN,NaN,1,41521_194908,CDU,1949-08-01,german,False,False,17734
3,"Der Kampf um die Ware, der ein Kampf gegen den...",NaN,NaN,1,41521_194908,CDU,1949-08-01,german,False,False,17734
4,Die Korruption griff in erschreckendem Maße um...,NaN,NaN,1,41521_194908,CDU,1949-08-01,german,False,False,17734
...,...,...,...,...,...,...,...,...,...,...,...
95678,Und zu machen.,202.1,NaN,4317,41113_202109,GREENS,2021-09-01,german,True,True,26929
95679,Jetzt liegt es bei Ihnen.,202.1,NaN,4318,41113_202109,GREENS,2021-09-01,german,True,True,26929
95680,In Wahlen entscheidet eine Gesellschaft darübe...,202.1,NaN,4319,41113_202109,GREENS,2021-09-01,german,True,True,26929
95681,Wahlen sind ein Moment der Freiheit.,202.1,NaN,4320,41113_202109,GREENS,2021-09-01,german,True,True,26929


In [8]:
df_manifesto_final.to_csv("data/parties/All_Manifestos.csv", index=True)